<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Marc Spiegelman, Template from Kyle Mandli</td>
</table>


# Mg2SiO4 Stixrude Equilibrium Phase diagram

This notebook will use functions from the ThermoCodegen generated Mg2SiO4 stixrude system to plot thermodynamic properties as a function of $T,P$ for the phase that satisfies $\min(G_i(T,P))$ for $i\in[0,2]$ 

In [ ]:
# load the standard goodies
import numpy as np
from scipy.optimize import fsolve, brentq
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline
plt.rcParams["figure.figsize"] = [8,6]
plt.rcParams['font.size'] = 16

### Load the ThermoCodegen object for the fo-fa reactive system

This will require that the python bindings for the py_fo_fa_binary module are in the python path

if you are using modules,  this can be accomplished by running
|
```bash
cd ../../reactions/Mg2SiO4_stixrude
module load ./Mg2SiO4.module


In [ ]:
import py_Mg2SiO4_stixrude as pms
#pms.phase_info()

In [ ]:
rxn = pms.Mg2SiO4_stixrude()
rxn.report()

utility functions 


In [ ]:
toCelsius = lambda T: T - 273.15
toKelvin = lambda T: T + 273.15
toBar = lambda P: P*10000.

### Plot Affinity Surfaces for the Fo-Wa reaction and Wa-Ri reaction

In [ ]:
ta = np.linspace(900.,2600.,100)
pa = np.linspace(.1,30.,100)
Tm,Pm = np.meshgrid(ta,pa)
C = rxn.zero_C()
for i in range(len(C)):
    C[i] = [1.]
    
A0 = np.zeros(Tm.shape)
A1 = np.zeros(Tm.shape)
for i,T in enumerate(ta):
    for j,P in enumerate(toBar(pa)):
        A = rxn.A(T,P,C)
        A0[j,i] = A[0]
        A1[j,i] = A[1]


## Phase Diagram

Contour 0 affinity for each reaction and compare to ThermoEngine

In [ ]:
fig = plt.figure(figsize=(4,8))
axes = fig.add_subplot(1,1,1)
axes.contour(Tm,Pm,A0,[0],colors = ['r'])
axes.contour(Tm,Pm,A1,[0],colors = ['r'])

axes.grid()
axes.set_xlabel('Temperature (K)')
axes.set_ylabel('Pressure (GPa)')
axes.invert_yaxis()
plt.show()

### Plot  $\min(G_i(T,P))$ and the index of the stable phase

In [ ]:
Gmin = np.zeros(Tm.shape)
index = np.zeros(Tm.shape, dtype=int)
for i,T in enumerate(ta):
    for j,P in enumerate(toBar(pa)):
        G = rxn.Mu(T,P,C)
        Gmin[j,i] = np.min(G)
        index[j,i]= np.argmin(G)
        

In [ ]:
fig = plt.figure(figsize=(10,8))
axes = fig.add_subplot(1,2,1)
cf = axes.contourf(Tm,Pm,Gmin)
axes.contour(Tm,Pm,A0,[0],colors = ['r'])
axes.contour(Tm,Pm,A1,[0],colors = ['r'])
axes.grid()
axes.set_xlabel('Temperature (K)')
axes.set_ylabel('Pressure (GPa)')
axes.invert_yaxis()
#plt.colorbar(cf)

axes = fig.add_subplot(1,2,2)
axes.contourf(Tm,Pm,index)

axes.grid()
axes.set_xlabel('Temperature (K)')
axes.set_ylabel('Pressure (GPa)')
axes.invert_yaxis()
plt.show()

### Plot  $\rho$, $\alpha$, $C_p$, $s$ for the stable phase

In [ ]:
rho = np.zeros(Tm.shape)
alpha = np.zeros(Tm.shape)
cp = np.zeros(Tm.shape)
s = np.zeros(Tm.shape)
for i,T in enumerate(ta):
    for j,P in enumerate(toBar(pa)):
        rho[j,i] = rxn.rho(T,P,C)[index[j,i]]
        alpha[j,i] = rxn.alpha(T,P,C)[index[j,i]]
        cp[j,i] = rxn.Cp(T,P,C)[index[j,i]]
        s[j,i] = rxn.s(T,P,C)[index[j,i]]
        

In [ ]:
fig = plt.figure(figsize=(14,16))
axes = fig.add_subplot(2,2,1)
cf = axes.contourf(Tm,Pm,rho/10)
axes.contour(Tm,Pm,A0,[0],colors = ['r'])
axes.contour(Tm,Pm,A1,[0],colors = ['r'])

axes.grid()
axes.set_title('$\\rho$ (g/cm$^3$)')
axes.set_xlabel('Temperature (K)')
axes.set_ylabel('Pressure (GPa)')
axes.invert_yaxis()
plt.colorbar(cf)

axes = fig.add_subplot(2,2,2)
cf = axes.contourf(Tm,Pm,alpha)
axes.contour(Tm,Pm,A0,[0],colors = ['r'])
axes.contour(Tm,Pm,A1,[0],colors = ['r'])
axes.grid()
axes.set_title('$\\alpha$ $(K^{-1})$')
axes.set_xlabel('Temperature (K)')
axes.set_ylabel('Pressure (GPa)')
axes.invert_yaxis()
plt.colorbar(cf)

axes = fig.add_subplot(2,2,3)
cf = axes.contourf(Tm,Pm,cp)
axes.contour(Tm,Pm,A0,[0],colors = ['r'])
axes.contour(Tm,Pm,A1,[0],colors = ['r'])
axes.grid()
axes.set_title('$C_p$ (J/gK)')
axes.set_xlabel('Temperature (K)')
axes.set_ylabel('Pressure (GPa)')
axes.invert_yaxis()
plt.colorbar(cf)

axes = fig.add_subplot(2,2,4)
cf = axes.contourf(Tm,Pm,s)
axes.contour(Tm,Pm,A0,[0],colors = ['r'])
axes.contour(Tm,Pm,A1,[0],colors = ['r'])
axes.grid()
axes.set_title('$s$  (J/gK)')
axes.set_xlabel('Temperature (K)')
axes.set_ylabel('Pressure (GPa)')
axes.invert_yaxis()
plt.colorbar(cf)

fig.subplots_adjust(wspace=0.2)
plt.show()

### Plot total entropy per-unit volume $\rho s$ and contour for isentropes

In [ ]:
fig = plt.figure(figsize=(6,8))
axes = fig.add_subplot(1,1,1)
cf = axes.contourf(Tm,Pm,rho*s)
axes.contour(Tm,Pm,A0,[0],colors = ['r'])
axes.contour(Tm,Pm,A1,[0],colors = ['r'])

axes.grid()
axes.set_title('$\\rho s$')
axes.set_xlabel('Temperature (K)')
axes.set_ylabel('Pressure (GPa)')
axes.invert_yaxis()
plt.colorbar(cf)
plt.show()

In [ ]:
### some reference values

iRi = 2
P0 = toBar(30.)
T0 = 2000.
rho0 = rxn.rho(T0,P0, C)[iRi]
s0 = rxn.s(T0, P0, C)[iRi]
print('Reference density rho_0={}, entropy={}'.format(rho0,s0))

In [ ]:
phases=rxn.phases()
phases[0].beta(T,P,[1.])*P0